<a href="https://colab.research.google.com/github/paulokuriki/jupyter_dicom_viewer/blob/main/DicomViewer_for_Jupyter_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Creating your a Python DICOM Viewer**

# 1. Installing necessary libraries

In [ ]:
#@title Executar configuração do ambiente { vertical-output: true, display-mode: "form" }
!pip install pydicom
!pip install pynetdicom

#!pip install ipywidgets

# 2. Declaring auxiliary functions

In [ ]:
import os
import glob
import requests
import sys
import zipfile

from IPython.display import clear_output, display
from ipywidgets import Button, FileUpload, VBox, Output, FileUpload, Tab, Dropdown, interactive_output, IntSlider, Select, ToggleButton
from tkinter import Tk, filedialog

import pydicom
import pynetdicom

from pandas import DataFrame
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# defining an empty dataframe
df = pd.DataFrame(columns=['patient_name', 'study_description', 'series_description'])

def download_file(url):
    local_filename = url.split('/')[-1]
    r = requests.get(url, allow_redirects=True)
    with open(local_filename, 'wb') as file:
        file.write(r.content)
        
def unzip_file(zip_file):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(os.getcwd())

In [ ]:
# Function download_samples()
# 
# Downloads sample dicom files from GitHub
   
def download_samples(b):

    with dicom_status_bar:
        clear_output()
        print('Downloading samples from GitHub...')
    
    download_file('https://github.com/paulokuriki/feres_python_2021/raw/main/torax.zip')
    unzip_file('torax.zip')
    os.remove('torax.zip')
    
    dicom_files = sorted(glob.glob('*.dcm'))
    with dicom_status_bar:
        print(f'Download finished. {len(dicom_files)} files downloaded.')
    
    # loads dicom files do dataframe
    load_dicom_files(dicom_files)

In [ ]:
# Function unzip_uploaded_files()
# 
# Unzip uploaded zip file

def upload_file(uploaded_file):
    with dicom_status_bar:
        clear_output()
        print('Function in development...')    

In [ ]:
# Function select_dicom_file() and select_dicom_folder()
#
# Selects Dicom Files or Dicom Folder paths

IN_COLAB = 'google.colab' in sys.modules

def select_dicom_files(b):

    if IN_COLAB:
        with dicom_status_bar:
            clear_output()
            print("Google Colab has no access to your local files. Try to run this feature locally.")
        return
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)
    dicom_files = filedialog.askopenfilename(multiple=True) 
    with dicom_status_bar:
        clear_output()
        print(f'{len(dicom_files)} files selected.')
    
    # loads dicom files do dataframe
    load_dicom_files(dicom_files)

def select_dicom_folder(b):
    if IN_COLAB:
        print("Google Colab has no access to your local files. Try to run this feature locally.")
        return
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    folder = filedialog.askdirectory()
    if folder:
        dicom_files = glob.glob(os.path.join(folder, '**/*',), recursive = True)
    else:
        dicom_files = []
    
    with dicom_status_bar:
        clear_output()
        print(f'{len(dicom_files)} files selected.')
        
    load_dicom_files(dicom_files)

In [ ]:
def query_pacs(b):
    
    with dicom_status_bar:
        clear_output()
        print(f'Function in development.')
    
    load_dicom_files([])

# 3: Loading DICOM files to the memory

Increases performance loading all images to the memory

In [ ]:
def load_dicom_files(dicom_files: list) -> DataFrame:
    global df
    
    dicom_images_list = []

    for filename in dicom_files:
        try:
            ds = pydicom.dcmread(filename)

            # converting to hounsfield if it is a ct
            if 'RescaleSlope' in ds:
                slope = float(ds.RescaleSlope)
                intercept = float(ds.RescaleIntercept)
                pixel_array = intercept + ds.pixel_array * slope
            else:
                pixel_array = ds.pixel_array

            dict_dicom = {}
            dict_dicom['filename'] = filename
            dict_dicom['patient_name'] = ds.PatientName
            dict_dicom['study_description'] = ds.StudyDescription
            dict_dicom['series_description'] = ds.SeriesDescription
            dict_dicom['study_uid'] = ds.StudyInstanceUID
            dict_dicom['series_uid'] = ds.SeriesInstanceUID
            dict_dicom['modality'] = ds.Modality
            dict_dicom['slice_location'] = ds.SliceLocation
            dict_dicom['pixel_array'] = ds.pixel_array

            dicom_images_list.append(dict_dicom)

        except Exception as e:
            # not a valid dicom file
            print(e)
            pass
        
    if len(dicom_images_list) > 0:
        df = pd.DataFrame(dicom_images_list)
        df = df.sort_values(by=['study_uid', 'series_uid', 'slice_location']).reset_index(drop=True)
    else:
        df = pd.DataFrame(columns=['patient_name', 'study_description', 'series_description'])

    with dicom_status_bar:
        print(f'{len(df)} DICOM files loaded to memory.')

# 4: Creating Function for Displaying DICOM Image

In [ ]:
def show_dicom_image(scroll, patient, study, series, window_level, window_width, zoom, predefined_window, save_to_file):
    
    if len(df) == 0:
        return
    
    try:
        # filters specific image
        img_pixel_array = df.loc[
           (df['patient_name'] == patient) & 
           (df['study_description'] == study) & 
           (df['series_description'] == series)
          ]['pixel_array'][scroll-1]
    except Exception as e:
        return
    
    if predefined_window == 'lung':
        window_level = -600
        window_width = 1500
    elif predefined_window == 'mediastinum':
        window_level = 50
        window_width = 350
    elif predefined_window == 'bone':
        window_level = 400
        window_width = 1800

    vmin = window_level - (window_width / 2)
    vmax = window_level + (window_width / 2)
    
    plt.figure(figsize=(zoom, zoom))
    plt.axis('off')
    
    with viewer_status_bar:
        clear_output()
        plt.imshow(img_pixel_array, vmin=vmin, vmax=vmax, cmap='gray')
        print(f"Slice: {scroll}     WL: {window_level}     WW: {window_width}     Zoom: {zoom}")
        
    if save_to_file:
        var_arq_dicom = dicom_images_list[scroll-1].get('filename','noname.dcm')
        var_nome_arq_jpg = var_arq_dicom.replace('.dcm', '.jpg')
        plt.savefig(var_nome_arq_jpg)
        print(f'Salvo para o arquivo: {var_nome_arq_jpg}')

    

# 5. Selecting DICOM Source

In [ ]:
#@title Selecting DICOM Source { display-mode: "form" }
    
# ----------- CLOUD DICOM SOURCE TAB ------------
tab_titles_dicom = ["Cloud DICOM source"]
btn_download_samples = Button(description="Download Samples", icon = "download", tooltip = "Download Dicom Sample Files from GitHub")
btn_download_samples.on_click(download_samples)
btn_upload_zip_file = FileUpload(description="Upload DICOM Zip", accept = '.zip', disabled = True, tooltip = "Uploads a Dicom zip File from your machine")
tab_contents_dicom = [VBox([btn_download_samples, btn_upload_zip_file])]

# ----------- LOCAL DICOM SOURCE TAB ------------
tab_titles_dicom.append("Local DICOM source")
btn_select_dicom_file = Button(description="Select DICOM File", icon = "file", disabled = IN_COLAB, tooltip = "Select one or more DICOM files from your machine")
btn_select_dicom_file.on_click(select_dicom_files)
btn_select_dicom_folder = Button(description="Select DICOM Folder", icon = "folder", disabled = IN_COLAB, tooltip = "Select a DICOM folder from your machine")
btn_select_dicom_folder.on_click(select_dicom_folder)
tab_contents_dicom.append(VBox([btn_select_dicom_file, btn_select_dicom_folder]))

# ----------- PACS QUERY / RETRIEVE TAB ------------
tab_titles_dicom.append("PACS Query/Retrieve")
btn_query_pacs = Button(description="Query/Retrive", icon = "server", disabled = IN_COLAB, tooltip = "Query and Retrive from a PACS")
btn_query_pacs.on_click(query_pacs)
tab_contents_dicom.append(VBox([btn_query_pacs]))

# --------------- CREATING TABS STRUCTURE ---------------
tab_dicom = Tab()
tab_dicom.children = tab_contents_dicom
for i in range(len(tab_contents_dicom)):
    tab_dicom.set_title(i, tab_titles_dicom[i])

# --------------- LOADING STATUS BAR -----------------------
dicom_status_bar = Output(layout={'border': '1px solid black'})
clear_output()
display(tab_dicom, dicom_status_bar)

Output(layout=Layout(border='1px solid black'))

In [ ]:
#@title Selecting Study { display-mode: "form" }
# creates buttons for select DICOM Source

# ----------- PATIENT TAB ------------
tab_titles_study="Select Study"
patient = Dropdown(options=sorted(list(df['patient_name'].drop_duplicates())), description='Patient:')
study = Dropdown(options=sorted(list(df['study_description'].drop_duplicates())), description='Study:')
series = Dropdown(options=sorted(list(df['series_description'].drop_duplicates())), description='Series:')
tab_contents_study = [VBox([patient, study, series])]

# --------------- CREATING TABS STRUCTURE ---------------
tab_study = Tab()
tab_study.children = tab_contents_study
tab_study.set_title(0, tab_titles_study)

# --------------- LOADING STATUS BAR -----------------------
clear_output()
study_status_bar = Output(layout={'border': '1px solid black'})
display(tab_study, study_status_bar)

Output(layout=Layout(border='1px solid black'))

In [ ]:
#@title Viewing Images { display-mode: "form" }
total_images = len(df) if len(df) > 0 else 1

# ------------ IMAGE TAB ---------------
tab_titles_viewer="Viewing Images"

scroll = IntSlider(value=54, min=1, max=total_images, step=1, description='Scroll:', continuous_update=False,
    orientation='horizontal', readout=True, readout_format='d')
window_level = IntSlider(value=-600, min=-1000, max=1000, step=1, description='Window Level:', continuous_update=False,
    orientation='horizontal', readout=True, readout_format='d')
window_width = IntSlider(value=1500, min=0, max=2000, step=1, description='Window Width:', continuous_update=True,
    orientation='horizontal', readout=True, readout_format='d')
zoom = IntSlider(value=10, min=1, max=10, step=1, description='Zoom:', continuous_update=False,
    orientation='horizontal', readout=True, readout_format='d')
predefined_window = Select(options=["custom", "lung", "mediastinum", "bone"], description='Window:')
save_to_file = ToggleButton(value=False, description='Save to File', tooltip='Save image locally to PNG', icon='save')

tab_contents_viewer = [VBox([scroll, window_level, window_width, zoom, predefined_window, save_to_file])]

# --------------- CREATING TABS STRUCTURE ---------------
tab_viewer = Tab()
tab_viewer.children = tab_contents_viewer
tab_viewer.set_title(0, tab_titles_viewer)

# --------------- LOADING STATUS BAR -----------------------
clear_output()
viewer_status_bar = Output(layout={'border': '1px solid black'})
display(tab_viewer, viewer_status_bar)

interactive_output(show_dicom_image, {'scroll': scroll, 'patient': patient, 'study': study, 'series': series, 'window_level': window_level, 'window_width': window_width, 
                                      'zoom': zoom, 'predefined_window': predefined_window, 'save_to_file': save_to_file})

Output(layout=Layout(border='1px solid black'))

Output()